In [1]:
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder
import json

In [6]:
def get_water_usage(objId, cluster, client_id, client_secret, authority_id, monthly = True):
    kcsb = KustoConnectionStringBuilder.with_aad_application_key_authentication(cluster, client_id, client_secret, authority_id)
    client = KustoClient(kcsb)
    db = "unity-consumption"
    
    if monthly:
        query = "consumption | extend Month=format_datetime(['time'],\"MM\") | summarize TotalConsumed = sum(Value) by ManagedObjectid, typeM, Month| where ManagedObjectid =="+str(objId)+" and (typeM ==\"Pulse1\" or typeM==\"/10266/0\")"
    else:
        query = "consumption | summarize TotalConsumed = sum(Value) by ManagedObjectid, typeM, bin(['time'], 7d) | where ManagedObjectid =="+str(objId)+" and (typeM ==\"Pulse1\" or typeM==\"/10266/0\")"
    response = client.execute(db, query)
    json_result = json.loads(str(response.primary_results[0]))
    return json_result['data']

In [10]:
get_water_usage(351, cluster, client_id, client_secret, authority_id, monthly = False)

[{'ManagedObjectid': 351,
  'typeM': 'Pulse1',
  'time': '2022-06-13 00:00:00+00:00',
  'TotalConsumed': 2444},
 {'ManagedObjectid': 351,
  'typeM': 'Pulse1',
  'time': '2022-06-20 00:00:00+00:00',
  'TotalConsumed': 3435},
 {'ManagedObjectid': 351,
  'typeM': 'Pulse1',
  'time': '2022-05-30 00:00:00+00:00',
  'TotalConsumed': 3442},
 {'ManagedObjectid': 351,
  'typeM': 'Pulse1',
  'time': '2022-06-06 00:00:00+00:00',
  'TotalConsumed': 3660},
 {'ManagedObjectid': 351,
  'typeM': 'Pulse1',
  'time': '2022-05-23 00:00:00+00:00',
  'TotalConsumed': 2640},
 {'ManagedObjectid': 351,
  'typeM': 'Pulse1',
  'time': '2022-05-02 00:00:00+00:00',
  'TotalConsumed': 3665},
 {'ManagedObjectid': 351,
  'typeM': 'Pulse1',
  'time': '2022-05-09 00:00:00+00:00',
  'TotalConsumed': 3182},
 {'ManagedObjectid': 351,
  'typeM': 'Pulse1',
  'time': '2022-05-16 00:00:00+00:00',
  'TotalConsumed': 3454},
 {'ManagedObjectid': 351,
  'typeM': 'Pulse1',
  'time': '2022-04-25 00:00:00+00:00',
  'TotalConsumed':